# Install libraries

In [ ]:
!pip install beautifulsoup4
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-driver

# Import libraries

In [ ]:
import pandas as pd
import numpy as np

# Prepare Driver

In [ ]:
from selenium import webdriver

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


# Login to investagrams

In [ ]:
url = "https://www.investagrams.com/Login?ret=https%3A%2F%2Fwww.investagrams.com%2FStock%2FPSE%3AJFC"
driver.get(url)
driver.implicitly_wait(10)

email = driver.find_element_by_css_selector("input[data-ng-model='LoginRequest.Username']")
email.clear()
email.send_keys("#######")

password = driver.find_element_by_css_selector("input[data-ng-model='LoginRequest.Password']")
password.clear()
password.send_keys("#######")

driver.find_element_by_class_name("investa-login--btn-login").click()

# Get all stock codes

In [ ]:
def get_all_stock_codes():
  url = "https://www.investagrams.com/Stock/RealTimeMonitoring"
  driver.get(url)
  driver.implicitly_wait(10)
  stock_cells = driver.find_elements_by_css_selector("#StockQuoteTable tr td:first-child a")
  return [cell.get_attribute("innerHTML") for cell in stock_cells]

# Get the historical data

In [ ]:
def get_historical_data(stock_code):
  # Get the table
  url = "https://www.investagrams.com/Stock/PSE:" + stock_code
  driver.get(url)
  driver.implicitly_wait(20)
  tables = driver.find_elements_by_tag_name("table")

  # Create temporary list to store values
  temp = []

  # Get the sector and subsector
  sector = tables[0].find_element_by_css_selector("tr:last-child td:nth-child(2) span").get_attribute("innerHTML")
  sub_sector = tables[1].find_element_by_css_selector("tr:last-child td:nth-child(2) span").get_attribute("innerHTML")

  # Retrieve the rows
  rows = tables[9].find_elements_by_tag_name("tr")

  # Map the values into the temporary list
  for row in rows:
    columns = row.find_elements_by_tag_name("td")
    temp.append([stock_code, sector, sub_sector] + [col.get_attribute("innerHTML") for col in columns])

  return temp

In [ ]:
# Retrieve stock historical data
data = []

codes = get_all_stock_codes()

for code in codes:
  print("Retrieving", code, "(" + str(codes.index(code) + 1) + "/" + str(len(codes)) + ")")
  try:
    data += get_historical_data(code)
  except:
    print("Failed to retrieve", code)

Retrieving AC (1/71)
Retrieving ACEN (2/71)
Retrieving ACEX (3/71)
Retrieving AGI (4/71)
Retrieving ALHI (5/71)
Retrieving ALLHC (6/71)
Retrieving ANS (7/71)
Retrieving APC (8/71)
Retrieving APL (9/71)
Retrieving APVI (10/71)
Retrieving APX (11/71)
Retrieving AREIT (12/71)
Retrieving AXLM (13/71)
Retrieving BC (14/71)
Retrieving BDO (15/71)
Retrieving BEL (16/71)
Retrieving BKR (17/71)
Retrieving BPI (18/71)
Retrieving BRN (19/71)
Retrieving C (20/71)
Retrieving CAT (21/71)
Retrieving CEB (22/71)
Retrieving CEI (23/71)
Retrieving CIC (24/71)
Retrieving CNPF (25/71)
Retrieving CNVRG (26/71)
Retrieving COL (27/71)
Retrieving COSCO (28/71)
Retrieving CPG (29/71)
Retrieving CPGP (30/71)
Retrieving CPM (31/71)
Retrieving CROWN (32/71)
Retrieving DD (33/71)
Retrieving DFNN (34/71)
Retrieving DIZ (35/71)
Retrieving DMC (36/71)
Retrieving DMW (37/71)
Retrieving DNL (38/71)
Retrieving DWC (39/71)
Retrieving ECP (40/71)
Retrieving EEI (41/71)
Retrieving EMP (42/71)
Retrieving EW (43/71)
Retrievi

# Export to csv

In [ ]:
# Store the data into a data frame
columns = ["Stock Code", "Sector", "Subsector", "Date", "Last Price", "Change", "%Change", "Open", "Low", "High", "Volume", "Net Foreign"]

# Create the data frame
df_data = pd.DataFrame(data, columns=columns)
df_data.to_csv("stocks_a.csv", index=False)
df_data